In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

#  Submit adsorption energy

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%aiida

# AiiDAlab imports.
import aiidalab_widgets_base as awb
from aiidalab_widgets_base.utils import list_to_string_range

# Custom imports.
from apps.surfaces.widgets.empa_viewer import EmpaStructureViewer

In [ ]:
Cp2kFragmentSeparationWorkChain = WorkflowFactory('nanotech_empa.cp2k.fragment_separation')

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()
structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureBrowserWidget(title="AiiDA database"),
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computer_code_dropdown = awb.ComputationalResourcesWidget(description="CP2K code:", input_plugin="cp2k")

In [ ]:
# Fragments.
fragment_molecule = ipw.Text(
    value='',
    placeholder='1 2 10..13',
    description='Molecule:',
    disabled=False
)

fragment_slab = ipw.Text(
    value='',
    placeholder='3 14..500',
    description='Substrate',
    disabled=False
)


# Multiplicities.
total_multiplicity = ipw.IntText(
    value=0,
    step=1,
    description='Total Multiplicity:',
    disabled=False
)

multiplicity_molecule = ipw.IntText(
    value=0,
    step=1,
    description='Molecule multiplicity:',
    disabled=False
)

multiplicity_slab = ipw.IntText(
    value=0,
    step=1,
    description='Slab multiplicity:',
    disabled=False
)


#Charge
charge = ipw.Text(
    value='0 0',
    placeholder='0 0',
    description='Charge of each fragment:',
    description_width='initial',
    disabled=False
)
#description
html_charge = ipw.HTML(
                    value="""
                <p style="font-weight:400;">Charge of each fragment.
                </p>
               """
                )

#UKS
uks = ipw.Checkbox(
    value=False,
    description='UKS',
    disabled=False,
    indent=False
)

#spin guess
spins_up = ipw.Text(
    value='',
    placeholder='1 2 10..13',
    description='Spins U:',
    disabled=False
)

spins_down = ipw.Text(
    value='',
    placeholder='3 4 14..17',
    description='Spins D:',
    disabled=False
)

fixed_atoms = ipw.Text(
    value='',
    placeholder='3 4 14..17',
    description='Fixed atoms:',
    disabled=False
)

#max nodes
max_nodes = ipw.IntText(
    value=48,
    description='Max # compute nodes',
    disabled=False,
    style={"description_width": "initial"}
)
#walltime
walltime = ipw.IntText(
    value=86400,
    description='Walltime seconds',
    disabled=False,
    style={"description_width": "initial"}
)
#protocol
protocol = ipw.Dropdown(     value='standard',
    description='Protocol:',
    disabled=False,       options = [("Standard", "standard"), ("Low accuracy", "low_accuracy"),("Debug","debug")])

In [ ]:
def get_builder_ads():
    """Get the builder for the adsorption energy calculation."""
    builder = Cp2kFragmentSeparationWorkChain.get_builder()
    builder.code = computer_code_dropdown.value
    builder.structure = structure_selector.structure_node

    # Fragments.
    builder.fragments = {
        'molecule': List(list=awb.utils.string_range_to_list(fragment_molecule.value)[0]),
        'slab': List(list=awb.utils.string_range_to_list(fragment_slab.value)[0]),
    }

    # Multiplicities.
    builder.multiplicities = {
        "total": Int(total_multiplicity.value),
        "molecule": Int(multiplicity_molecule.value),
        "slab": Int(multiplicity_slab.value),
        }

    builder.options = {
        'total': {
            "max_wallclock_seconds": 600,
            "resources": {
                "num_machines": 1,
                "num_mpiprocs_per_machine": 4,
            },
        },
        'molecule': {
            "max_wallclock_seconds": 600,
            "resources": {
                "num_machines": 1,
                "num_mpiprocs_per_machine": 1,
            },
        },
        'slab': {
            "max_wallclock_seconds": 600,
            "resources": {
                "num_machines": 1,
                "num_mpiprocs_per_machine": 4,
            },
        },
    }


    # spin guess
    # ase_geom =  structure_selector.structure
    # mag_list = [ 0 for t in ase_geom ]
    # if uks.value:
    #     for i in awb.utils.string_range_to_list(spins_up.value)[0]:
    #         mag_list[i] = 1
    #     for i in awb.utils.string_range_to_list(spins_down.value)[0]:
    #         mag_list[i] = -1 
            
        
    #builder.magnetization_per_site = List(list=mag_list)


    builder.fixed_atoms = Str('3..18')
    #Charge
    builder.protocol = Str(protocol.value)


    mag = [0 for i in structure_selector.structure]
    builder.magnetization_per_site = List(list=mag)

    #builder.walltime_seconds = Int(walltime.value)
    #builder.max_nodes = Int(max_nodes.value)    
    
    return builder

In [ ]:
def after_submission(_=None):   
    structure_selector.value = None

btn_submit_ads = awb.SubmitButtonWidget(Cp2kFragmentSeparationWorkChain, 
                                input_dictionary_function=get_builder_ads
                               )
btn_submit_ads.btn_submit.disabled=True
btn_submit_ads.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit_ads.btn_submit.disabled=False
    #check system
    mol_on_surf = empa_viewer.details['system_type'] == 'SlabXY'  
    mol_on_surf = mol_on_surf and  len(empa_viewer.details['all_molecules'])>0    
    msg = 'Adsorption energy for this system not implemented'
    if mol_on_surf:
        btn_submit_ads.btn_submit.disabled=False  
        msg = ''
    spins_up.value = awb.utils.list_to_string_range(empa_viewer.details['spins_up'])
    spins_down.value = awb.utils.list_to_string_range(empa_viewer.details['spins_down'])
    with output:
        clear_output()
        print(msg)
        to_display = [ipw.HBox([fragment_molecule, fragment_slab])]
        if uks.value:
            to_display +=  [charge, fixed_atoms, uks, spins_up, spins_down, ipw.HBox([whole_multiplicity, multiplicity_molecule, multiplicity_slab]),
                           ipw.HBox([multiplicity,html_mult]),protocol]
        else:
            to_display +=  [charge,fixed_atoms, uks,protocol]
        to_display.append(walltime)
        to_display.append(max_nodes)
        to_display.append(computer_code_dropdown)
        to_display.append(btn_submit_ads)
        display(ipw.VBox(to_display))


structure_selector.observe(update_all, names='structure')
uks.observe(update_all, names='value')

In [ ]:
display(output)